In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tiny-gpt-corpus/Tiny_data.txt


In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

with open('/kaggle/input/tiny-gpt-corpus/Tiny_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
import sentencepiece as spm
# 从TXT文件训练BPE模型
spm.SentencePieceTrainer.train(input='/kaggle/input/tiny-gpt-corpus/Tiny_data.txt', model_prefix='bpe_model', vocab_size=10000, model_type='bpe')
## load the BPE embedding model
sp = spm.SentencePieceProcessor()
sp.load('bpe_model.model')

#get the vocab ids 
token_ids = sp.EncodeAsIds(text)

# test
print(sp.DecodeIds(token_ids)[:100])

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /kaggle/input/tiny-gpt-corpus/Tiny_data.txt
  input_format: 
  model_prefix: bpe_model
  model_type: BPE
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  diffe

First Citizen: Before we proceed any further, hear me speak. All: Speak, speak. First Citizen: You a


92
trainer_interface.cc(591) LOG(INFO) Done! preprocessed 32777 sentences.
trainer_interface.cc(597) LOG(INFO) Tokenizing input sentences with whitespace: 32777
trainer_interface.cc(608) LOG(INFO) Done! 25670
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=24133 min_freq=1
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=6114 size=20 all=1626 active=1566 piece=it
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=3906 size=40 all=2197 active=2137 piece=st
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=2392 size=60 all=2967 active=2907 piece=ld
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1864 size=80 all=3580 active=3520 piece=ke
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1429 size=100 all=4127 active=4067 piece=▁his
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=1396 min_freq=99
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=1253 size=120 all=4692 active=1564 piece=al
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=990 size=140 all=53

In [3]:
class GPTconfig:
    #define dropout rate for each part of the GPT
    embed_dropout=0.1
    att_dropout=0.1
    ff_dropout=0.1
    def __init__(self,vocab_size,max_len,**kwargs):
        self.vocab=vocab_size
        self.max_len=max_len
        for k,v in kwargs.items():
            setatter(self,k,v)
class GPT2_config(GPTconfig):
    num_heads=12
    num_blocks=12
    embed_dim=768
    vocab_size = 10000
vocab_size = 10000
max_len = 32
config = GPT2_config(vocab_size, max_len)
   

In [4]:
class GPT_2(nn.Module):
    def __init__(self,config):
        super().__init__()
        embed_dim=config.embed_dim
        self.max_len=config.max_len
        
        self.token_embed=nn.Embedding(config.vocab_size,embed_dim)
        self.pos_embed=nn.Parameter(torch.zeros(1,config.max_len,config.embed_dim))## position embedding ?????
        self.dropout=nn.Dropout(config.embed_dropout)
        self.blocks=nn.Sequential(*[Transformer_block(config) for i in range(config.num_blocks)])
        self.ln=nn.LayerNorm(embed_dim)
        self.fc=nn.Linear(embed_dim,config.vocab_size)
    def forward(self,x,targets=None):
        input_len=x.size(1)
        assert input_len<=self.max_len,"seq longer than the maximum length"
        tok_embedding=self.token_embed(x)
        pos_embedding=self.pos_embed[:,:input_len,:]
        x=tok_embedding+pos_embedding
        x=self.dropout(x)
        x=self.blocks(x)
        x=self.ln(x)
        logits=self.fc(x)
        
        if targets is None:
            loss=None
        else:
            B,T,C=logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

In [5]:
class Transformer_block(nn.Module):
    def __init__(self,config):
        super().__init__()
        embed_dim=config.embed_dim
        self.ln_for_att=nn.LayerNorm(embed_dim)
        self.ln_for_ff=nn.LayerNorm(embed_dim)
        self.att=MultiHead_Attention(config)
        self.ff=nn.Sequential(
            nn.Linear(embed_dim,embed_dim*4),
            nn.GELU(),#### nice actiavte function for the transformer achitecture
            nn.Linear(embed_dim*4,embed_dim),
            nn.Dropout(config.ff_dropout)
        )
    def forward(self,x):
        x=self.att(self.ln_for_att(x))
        x=self.ff(self.ln_for_ff(x))
        return x

In [6]:
class Self_attention(nn.Module):
    def __init__(self,config,head_size):
        super().__init__()
        embed_dim=config.embed_dim
        block_size=config.max_len
        self.key=nn.Linear(embed_dim,head_size)
        self.value=nn.Linear(embed_dim,head_size)
        self.query=nn.Linear(embed_dim,head_size)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
        self.dropout=nn.Dropout(config.att_dropout)
    
    def forward(self,x):
        B,T,C=x.shape# Batch,Time step(token),Dimension of each token vector
        k=self.key(x)
        q=self.query(x)
        v = self.value(x)
        w=q@k.transpose(-2,-1) # transpose the last 2 dimension, bacase 1 is batch,but 2 and 3 is the matrix for tokens: Q*K'
        w=w*C**-0.5 # scaled
        w=w.masked_fill(self.tril[:T, :T] == 0, float('-inf')) ## using the trill to mask the token after t token.
        w = F.softmax(w, dim=-1)
        w = self.dropout(w)
        out = w @ v
        return out

In [7]:
class MultiHead_Attention(nn.Module):
    def __init__(self,config):
        super().__init__()
        head_size=config.embed_dim//config.num_heads
        num_heads=config.num_heads
        embed_dim=config.embed_dim
        self.heads=nn.ModuleList([Self_attention(config,head_size) for _ in range(num_heads)])
        self.proj=nn.Linear(embed_dim,embed_dim)
        self.dropout=nn.Dropout(config.att_dropout)
        
    def forward(self,x):
        out=torch.cat([h(x) for h in self.heads],dim=-1)
        out=self.dropout(self.proj(out))
        return out
        
        
    

In [8]:
# # hyperparameters
# batch_size = 16 # how many independent sequences will we process in parallel?
# block_size = 32 # what is the maximum context length for predictions?
# max_iters = 5000
# eval_interval = 100
# learning_rate = 1e-3
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# eval_iters = 200
# n_embd = 64
# n_head = 4
# n_layer = 4
# dropout = 0.0
# Hyperparameters
hyperparams = {
    "batch_size": 16, # how many independent sequences will we process in parallel?
    "block_size": 32, # what is the maximum context length for predictions?
    "max_iters": 5000,
    "eval_interval": 100,
    "learning_rate": 1e-3,
    "device": 'cuda' if torch.cuda.is_available() else 'cpu',
    "eval_iters": 200,
    "n_embd": 64,
    "n_head": 4,
    "n_layer": 4,
    "dropout": 0.0
}


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
# Prepare the data
data = torch.tensor(token_ids, dtype=torch.long)
train_data = data

def get_batch():
    ix = torch.randint(len(train_data) - hyperparams['block_size'], (hyperparams['batch_size'],))
    x = torch.stack([train_data[i:i+hyperparams['block_size']] for i in ix])
    y = torch.stack([train_data[i+1:i+hyperparams['block_size']+1] for i in ix])
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {'train': 0}
    model.eval()
    losses = torch.zeros(hyperparams['eval_iters'], device=hyperparams['device'])
    for k in range(hyperparams['eval_iters']):
        X, Y = get_batch()
        X, Y = X.to(hyperparams['device']), Y.to(hyperparams['device'])
        _, loss = model(X, Y)
        losses[k] = loss.item()
    out['train'] = losses.mean().item()
    model.train()
    return out

def train_model(model, token_ids, hyperparams):
    model.to(hyperparams['device'])
    optimizer = optim.AdamW(model.parameters(), lr=hyperparams['learning_rate'])

    for iter in range(hyperparams['max_iters']):
        if iter % hyperparams['eval_interval'] == 0 or iter == hyperparams['max_iters'] - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}")

        xb, yb = get_batch()
        xb, yb = xb.to(hyperparams['device']), yb.to(hyperparams['device'])
        _, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

# Initialize model
model = GPT_2(config)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')



# start training
train_model(model, token_ids, hyperparams)

100.450576 M parameters
step 0: train loss 9.4350
step 100: train loss 6.7857
step 200: train loss 6.6965
step 300: train loss 6.6773
step 400: train loss 6.6275
step 500: train loss 6.6190
step 600: train loss 6.6343
step 700: train loss 6.6059
step 800: train loss 6.5875
step 900: train loss 6.5868
step 1000: train loss 6.5991
step 1100: train loss 6.5985
step 1200: train loss 6.5973
step 1300: train loss 6.6189
step 1400: train loss 6.6048
step 1500: train loss 6.6009
step 1600: train loss 6.5850
step 1700: train loss 6.5856
step 1800: train loss 6.6001
step 1900: train loss 6.6076
step 2000: train loss 6.5844
step 2100: train loss 6.5980
step 2200: train loss 6.5728
step 2300: train loss 6.5962
step 2400: train loss 6.5860
step 2500: train loss 6.5841
step 2600: train loss 6.6091
step 2700: train loss 6.5824
step 2800: train loss 6.5992
step 2900: train loss 6.6056
step 3000: train loss 6.5823
step 3100: train loss 6.5923
step 3200: train loss 6.5979
step 3300: train loss 6.5951
st